In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from abc import ABC, abstractmethod
import random
import gc

#from ModelUtility import square as square

In [ ]:
#square(12)

Workflow:
1. Initialize Board
2. Initialize SpeciesList
3. Initialize some Species

Ideal usage:
```
sim = Simulation(100, 10000) # Board size (100x100), timesteps 10000
sim.loadParameters("params.txt")
sim.runSimulation()
sim.printAllSpecies()
```

In [ ]:
# DO NOT DELETE
allSpecies = {}
# Format:
# [t1, t2, t3, a1, a2, a3, b1, b2, b3, f1, f2, f3 speciationRate, maximumAge, maximumAgeDeviance]
# t1, t2, t3: Length of seed, juvenile, and adult stage
# a1, a2, a3: CNDD effects on seed, juvenile, and adult stage
# b1, b2, b3: HNDD effects on seed, juvenile, and adult stage
# f1, f2, f3: Fitness at seed, juvenile, and adult stage
# speciationRate: speciationRate
# maximumAge, maximumAgeDeviance: death age is normally distributed with mean maximumAge and standard deviation maximumAgeDeviance
# seedRate, seedDeviance: seeds produced per tick is normally distributed with mean seedRate and standard deviation seedDeviance
allSpecies[1] = [5, 10, 12, 0, 0, 0, 0, 0, 0, 1/3, 1/3, 1/3, 20, 0, 48, 12]
allSpecies[1]

In [ ]:
class Species:
    # 90% of the population will be in maximumAge +/- maximumAgeDeviance
    def __init__(self, t1, t2, t3, a1, a2, a3, b1, b2, b3, f1, f2, f3, maximumAge, maximumAgeDeviance):
        self.t1 = t1
        self.t2 = t2
        self.t3 = t3
        self.a1 = a1
        self.a2 = a2
        self.a3 = a3
        self.b1 = b1
        self.b2 = b2
        self.b3 = b3
        self.f1 = f1
        self.f2 = f2
        self.f3 = f3
        self.maximumAge = maximumAge
        self.maximumAgeDeviance = maximumAgeDeviance / 1.645
        
    #def 

The `Seed`, `Juvenile`, and `Adult` classes all inherit from the `Plant` parent class. A `Plant` must have a `speciesID`, a `maximumAge` determined by their species, and their `currentAge`. Every `Plant` has an `updateTick` method, where they either grow older or die. In the event that a `Seed` or a `Juvenile` lives, we also check if they can grow into their next stage of life. Meanwhile, for `Adult` plants, we check their dispersal of seeds.

In [6]:
"""
Simulation class.

Methods:
__init__

runSimulation
-> updateBoard
-> saveBoard

updateBoard
-> killBoard (requires Plant.willDie())
-> growBoard (requires Plant.grow())

printBoard
getBoard
resetBoard
saveBoard(filepath)
saveParameters(filepath)
loadBoard(filepath)
loadParameters(filepath)
"""

class Simulation:
    def __init__(self, boardLength: int, maximumTime: int):
        self.board = [[Adult(random.randint(0, 30), random.randint(0, 100), 0, 10, 20) for _ in range(boardLength)] for _ in range(boardLength)]
        # Board should be initialized filled with SeedArray
        self.boardLength = boardLength
        self.maximumTime = maximumTime
        self.totalSpeciesCount = 30
        self.currentTime = 0
        
    def runSimulation(self):
        while (self.currentTime < self.maximumTime):
            self.updateBoard() # Main step, time update is in here
            if (self.currentTime % 1000 == 0):
                filename = "timestep_" + str(self.currentTime) + ".csv"
                self.saveBoard(filename)
        filename = "timestep_" + str(self.currentTime) + ".csv"
        self.saveBoard(filename)
        return "Simulation finished."
        
    ##########################################################
    def updateBoard(self):
        self.killBoard()
        self.growBoard()
        self.currentTime += 1
        
    def killBoard(self):
        for i in range(self.boardLength):
            for j in range(self.boardLength):
                # Every board element is either an Adult or a PlantArray
                newPlant = self.board[i][j].update() # PlantArray, Adult, or None
                if (type(newPlant) == Adult):
                    self.board[i][j] = newPlant
                elif (type(self.board[i][j] == Adult)):
                    self.board[i][j] = PlantArray() # Place a new SeedArray if the old tree dies
#                    if (type(self.board[i][j]) == Adult): # Updates adults
#                        self.board[i][j].something()
#                    elif (type(self.board[i][j] == PlantArray)): # Updates seeds and juveniles
#                        newPlantArray = self.board[i][j].update()
#                        if (type(newPlantArray) == Adult):
#                            self.board[i][j] = newPlantArray
        gc.collect() # Allegedly deletes all the objects which no longer have references
        
    def spawnSeeds(self, x, y):
        numbers = [-2, -1, 0, 1, 2]
        availableTiles = []
        for i in numbers:
            for j in numbers:
                if (type(self.board[x + i][y + j]) == PlantArray):
                    availableTiles.append(self.board[x + i][y + j])
        for PA in availableTiles:
            PA.merge(self.board[x][y].makePlantArray())
    
    def growBoard(self): # Only needs to plant seeds
        for i in range(self.boardLength):
            for j in range(self.boardLength):
                if (type(self.board[i][j]) == Adult):
                    self.spawnSeeds(i, j)
    
    def populateBoard(self):
        # Starts the population of the board
        return 0
    ##########################################################
    
    ##########################################################    
    # INTERNAL USE
    def returnSpeciesID(self):
        positions = [pd.DataFrame(columns=['x', 'y']) for i in range(self.totalSpeciesCount)]
        for i in range(self.boardLength):
            for j in range(self.boardLength):
                if (type(self.board[i][j]) == Adult):
                    speciesID = self.board[i][j].getSpeciesID()
                    positions[speciesID - 1].loc[len(positions[speciesID - 1])] = {'x': i, 'y': j}
        # Returns a list of dataframes storing the (x,y) coordinates of the adults.
        return positions
    
    # EXTERNAL USE
    def printBoard(self):
        printedBoard = [[0 for _ in range(self.boardLength)] for _ in range(self.boardLength)]
        for i in range(self.boardLength):
            for j in range(self.boardLength):
                if (type(self.board[i][j]) == Adult):
                    printedBoard[i][j] = self.board[i][j].getSpeciesID()
        plt.imshow(printedBoard, cmap='hot', interpolation='nearest')
        plt.colorbar()
        plt.show()
        return "Visualization with the \"hot\" heatmap."
    
    # EXTERNAL USE
    def printSpeciesID(self, speciesID: int):
        printedBoard = [[1 for _ in range(self.boardLength)] for _ in range(self.boardLength)]
        for i in range(self.boardLength):
            for j in range(self.boardLength):
                if (type(self.board[i][j]) == Adult):
                    if (self.board[i][j].getSpeciesID() == speciesID):
                        printedBoard[i][j] = 0
        plt.imshow(printedBoard, cmap='hot', interpolation='nearest')
        plt.show()
        return "Visualization with the \"hot\" heatmap."
    
    # EXTERNAL USE
    def printAllSpecies(self):
        speciesCount = 30 # Change this
        rowCount = int(np.sqrt(speciesCount))
        colCount = int(np.ceil(speciesCount / rowCount))
        fig, axes = plt.subplots(rowCount, colCount, figsize=(15,15))
        axes = axes.flatten()
        positions = self.returnSpeciesID()
        for i in range(speciesCount):
            axes[i].scatter(positions[i]['x'], positions[i]['y'], c='black')
            axes[i].set_title(f'Species {i + 1}')
        plt.tight_layout()
        plt.show()
    ##########################################################

    ##########################################################
    def getBoard(self):
        return self.board

    # TESTING USE
    def resetBoard(self):
        for i in range(self.boardLength):
            for j in range(self.boardLength):
                self.board[i][j] = 0
    
    # EXTERNAL USE
    def saveBoard(self, filepath):
        columnNames = ["x", "y", "speciesID", "maximumAge", "currentAge", "type"]
        allPlants = []
        for i in range(self.boardLength):
            for j in range(self.boardLength):
                item = self.board[i][j]
                if (type(item) == Adult):
                    allPlants.append(
                        [i, j, item.getSpeciesID(), item.getMaximumAge(), item.getCurrentAge(), 2]
                    )
                elif (type(item) == Juvenile):
                    allPlants.append(
                        [i, j, item.getSpeciesID(), item.getMaximumAge(), item.getCurrentAge(), 1]
                    )
                elif (type(item) == PlantArray):
                    seedArray = item.displaySeeds()
                    for k in range(len(seedArray)):
                        allPlants.append(
                            [i, j, seedArray[k].getSpeciesID(), seedArray[k].getMaximumAge(), seedArray[k].getCurrentAge(), 0]
                        )
        boardAsDataFrame = pd.DataFrame(allPlants, columns=columnNames)
        try:
            boardAsDataFrame.to_csv(filepath, index=False)
            return "File saved successfully."
        except:
            return "Error saving to file! Please provide the full filepath e.g., \"test.csv\" "
    
    # TODO
    def saveParameters(self, filepath):
        # TODO
        return 0
    
    # EXTERNAL USE
    def loadBoard(self, filepath):
        loadedData = pd.read_csv(filepath)
        seedArray = []
        for _, row in loadedData.iterrows():
            if (row['type'] == 2):
                self.board[row['x']][row['y']] = Adult(
                    row['speciesID'],
                    row['maximumAge'],
                    row['currentAge']
                )
            elif (row['type'] == 1):
                self.board[row['x']][row['y']] = Juvenile(
                    row['speciesID'],
                    row['maximumAge'],
                    row['currentAge']
                )
            elif (row['type'] == 0):
                if (type(self.board[row['x']][row['y']]) == SeedArray):
                    self.board[row['x']][row['y']].addSeed(
                        Seed(row['speciesID'], row['maximumAge'], row['currentAge'])
                    )
                else:
                    self.board[row['x']][row['y']] = SeedArray()
                    self.board[row['x']][row['y']].addSeed(
                        Seed(row['speciesID'], row['maximumAge'], row['currentAge'])
                    )
        return "File loaded successfully."
    
    # TODO
    def loadParameters(self, filepath):
        # TODO
        params = pd.read_csv(filepath)
        return 0
    ##########################################################

In [ ]:
%%time
sim = Simulation(100, 1000)
sim.saveBoard("test1.csv")

In [ ]:
%%time
sim.runSimulation()

In [ ]:
sim.resetBoard()

In [ ]:
%%time
sim.getBoard()

In [ ]:
%%time
sim.printSpeciesID(5)

In [ ]:
%%time
sim.printAllSpecies()

In [ ]:
sim.loadBoard("test1.csv")

In [2]:
"""
Plant class.

Methods:
__init__

getCurrentAge ✔️
getMaximumAge ✔️
getSpeciesID ✔️

willDie (requires allSpecies dict to be created)
grow
becomeJuvenile (Seed)
becomeAdult (Juvenile)
createSeeds (Adult)
"""

class Plant:
    def __init__(self, speciesID, maximumAge, currentAge, seedLength, juvenileLength):
        self.speciesID = speciesID
        self.maximumAge = maximumAge
        self.currentAge = currentAge
        self.seedLength = seedLength
        self.juvenileLength = juvenileLength
        
    def getCurrentAge(self) -> int:
        return self.currentAge
    
    def getMaximumAge(self) -> int:
        return self.maximumAge
    
    def getSpeciesID(self) -> int:
        return self.speciesID
    
    def getSeedLength(self) -> int:
        return self.seedLength
    
    def getJuvenileLength(self) -> int:
        return self.juvenileLength
    
    #@abstractmethod
    #def willDie(self) -> bool:
    #    pass
    
    #@abstractmethod
    #def successfulTransition(self) -> bool:
    #    pass

class Seed(Plant):
    def __init__(self, speciesID: int, maximumAge: int, currentAge: int, seedLength: int, juvenileLength: int):
        super().__init__(speciesID, maximumAge, currentAge, seedLength, juvenileLength)
        
    def successfulTransition(self) -> bool:
        return random.random() < allSpecies[1][9] # Change the 1 to self.speciesID, remove hardcoding
        
    # Increments age, checks if it is too old, then checks for transition, then returns self.
    def grow(self) -> Plant:
        self.currentAge += 1
        if (self.currentAge > self.maximumAge):
            return None
        elif (self.currentAge >= allSpecies[1][0]): # Change the first 1 to self.speciesID, remove hardcoding
            if (self.successfulTransition()):
                return Juvenile(
                    self.speciesID, self.maximumAge, self.currentAge, self.seedLength, self.juvenileLength
                )
            else:
                return None
        else:
            return self
            
class Juvenile(Plant):
    def __init__(self, speciesID: int, maximumAge: int, currentAge: int, seedLength: int, juvenileLength: int):
        super().__init__(speciesID, maximumAge, currentAge, seedLength, juvenileLength)
    
    def willDie(self) -> bool: # Dies if True
        if (self.currentAge > self.maximumAge):
            return True
        return False
    
    def successfulTransition(self) -> bool:
        return random.random() < allSpecies[1][10] # Change the 1 to self.speciesID, remove hardcoding
    
    # Increments age, checks if it is too old, then checks for transition, then returns self.
    def grow(self) -> Plant:
        self.currentAge += 1
        if (self.currentAge > self.maximumAge):
            return None
        elif (self.currentAge >= allSpecies[1][1]): # Change the first 1 to self.speciesID, remove hardcoding
            if (self.successfulTransition()):
                return Adult(
                    self.speciesID, self.maximumAge, self.currentAge, self.seedLength, self.juvenileLength
                )
            else:
                return None
        else:
            return self

class Adult(Plant):
    def __init__(self, speciesID: int, maximumAge: int, currentAge: int, seedLength: int, juvenileLength: int):
        super().__init__(speciesID, maximumAge, currentAge, seedLength, juvenileLength)
        self.isFruiting = False
    
    def update(self): # Returns Adult if alive and None if dead
        self.currentAge += 1
        if (self.currentAge > self.maximumAge):
            return None
        elif (self.currentAge >= allSpecies[1][2]): # Change the first 1 to self.speciesID, remove hardcoding
            self.isFruiting = True
            if (self.successfulTransition()):
                return self
            else:
                return None
    
    def successfulTransition(self) -> bool: # Dies if True
        return random.random() < allSpecies[1][11] # Change the 1 to self.speciesID, remove hardcoding
        
    def makePlantArray(self):
        seedCount = round(random.normal(allSpecies[1][14] / 20, allSpecies[1][14] / 20)) # Change the 1 to self.speciesID, remove hardcoding
        returnPlantArray = PlantArray()
        for seed in range(seedCount):
            returnPlantArray.addSeed(Seed(self.speciesID, 100, 0, allSpecies[1][0], allSpecies[1][1])) # Remove hardcoding
        return returnPlantArray
        
class PlantArray: # Contains both seeds and juveniles
    def __init__(self):
        self.plantArray = []
        self.adultArray = []
        
    def addSeed(self, seed: Seed):
        self.plantArray.append(seed)
        
    def update(self): # Either returns Adult or None
        if (len(self.adultArray) != 0):
            index = random.randint(0, len(self.adultArray) - 1)
            return self.adultArray[index]
        else:
            for i in self.plantArray:
                newPlant = i.grow()
                if (newPlant is None):
                    self.plantArray.remove(i)
                elif (type(newPlant) == Adult):
                    self.adultArray.append(newPlant)
                else:
                    self.plantArray[self.plantArray.index(i)] = newPlant
            return None
        
    def merge(self, newPlantArray):
        self.plantArray.append(newPlantArray.displayP())
        self.adultArray.append(newPlantArray.displayAdults())
        
    # TESTING USE
    def addAdult(self, adult: Adult):
        self.adultArray.append(adult)
            
    def displaySeeds(self):
        returnList = []
        for i in self.plantArray:
            returnList.append(i.getSpeciesID())
        return returnList
    
    def displayP(self):
        return self.plantArray
    
    def displayAdults(self):
        return self.adultArray

In [ ]:
class SpeciesList:
    def __init__(self):
        self.speciesList = {}
        
    def addSpecies(self, species):
        self.speciesList[species[0]] = species

In [ ]:
testPlantArray = PlantArray()
testPlantArray.addSeed(Seed(10, 100, 0, 10, 20))
print(testPlantArray.displaySeeds())
testPlantArray.addSeed(Seed(12, 100, 0, 10, 20))
testPlantArray.addSeed(Seed(15, 100, 0, 10, 20))
testPlantArray.addSeed(Seed(18, 100, 0, 10, 20))
testPlantArray.addSeed(Juvenile(1, 100, 0, 10, 20))
print(testPlantArray.displaySeeds())
print(testPlantArray.displayAdults())
testPlantArray.addAdult(Adult(42, 100, 0, 10, 20))
print(testPlantArray.displayAdults())

In [ ]:
tPA = PlantArray()
tPA.addSeed(Seed(1, 10, 0, 3, 5))
tPA.addSeed(Seed(2, 10, 0, 3, 5))
tPA.addSeed(Seed(3, 10, 0, 3, 5))
tPA.addSeed(Seed(4, 10, 0, 3, 5))
tPA.addSeed(Seed(5, 10, 0, 3, 5))
print(tPA.displayP())
tPA.update()
print(tPA.displayP())
tPA.update()
print(tPA.displayP())
tPA.update()
print(tPA.displayP())
tPA.update()
print(tPA.displayP())
tPA.update()
print(tPA.displayP())
tPA.update()
print(tPA.displayP())
tPA.update()
print(tPA.displayP())
tPA.update()
print(tPA.displayP())
tPA.update()
print(tPA.displayP())
print(tPA.displayAdults())
tPA.update()
print(tPA.displayP())
print(tPA.displayAdults())
tPA.update()
print(tPA.displayP())
print(tPA.displayAdults())
tPA.update()
print(tPA.displayP())
print(tPA.displayAdults())

In [ ]:
simList = []
simList.append(testPlantArray)
print(simList)
for i in simList:
    i.update()
print(simList)
print(simList[0].displayAdults())

In [ ]:
seed = Seed(10, 0)
seed.getCurrentAge()
seed.getMaximumAge()

seedArray = SeedArray()
for i in range(5):
    seedArray.addSeed(Seed(i, 10))
    
seedArray.displaySeeds()

In [ ]:
plant = Plant(4, 1, 1)
plant.getSpeciesID()

In [ ]:
plant == 0

In [ ]:
seed2 = Seed(3, 10, 2)
seed2.getSpeciesID()

In [ ]:
seed2.getCurrentAge()

In [ ]:
# TO DO

import unittest

class TestNotebook(unittest.TestCase):
    
#    def test_add(self):
#        self.assertEqual(square(2), 4)
        
    def test_seed(self):
        seed = Seed(1, 2, 3, 4, 5)
        self.assertEqual(seed.getSpeciesID(), 1)
        
unittest.main(argv=[''], verbosity=2, exit=False)

In [ ]:
# OLD CODE
class SeedArray:
    def __init__(self):
        self.seedArray = []
        
    def addSeed(self, seed: Seed):
        self.seedArray.append(seed)
        
    def update(self):
        for i in self.seedArray:
            i.update()
            
    def displaySeeds(self):
        returnList = []
        for i in self.seedArray:
            returnList.append(i.getSpeciesID())
        return returnList